# Exercise 2 - One-vs-all MNIST
## Imports

In [7]:
import numpy as np
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
import pickle
import os
from tabulate import tabulate
print("Current working directory:", os.getcwd(), sep="\n")

# So that changes to the a3 model are reflected here.
import a3
import importlib
importlib.reload(a3)
import a3


Current working directory:
/Users/atakancoban/Desktop/School/2dv506 - Machine learning/Assignment 3/Lecture_8


## Functions

In [8]:
def predict(clfs, test):
    preds = []
    for i in range(len(clfs)):
        print(clfs[i].predict(test))
        if clfs[i].predict(test) == 1:
            preds.append(i)

def save_pickle(file_path, data):
    file = open(file_path, 'ab')
    pickle.dump(data, file)
    file.close()

def load_pickle(file_path):
    file = open(file_path, 'rb')
    data = pickle.load(file)
    file.close()
    return data

## Part 1 - Load & trim MNIST dataset

In [9]:
X_train, y_train, X_test, y_test = a3.mnist()

# Trim data.
trim = True
if trim:
    train_size = 10000
    test_size = 10000
    X_train, y_train = X_train[:train_size, :], y_train[:train_size, :]
    X_test, y_test = X_test[:test_size, :], y_test[:test_size, :]

## Part 2 - Train or load SVMs for each class

In [10]:
# Loading makes results fail on my laptop atm.
# True to load from disk, False to train models.
load_clfs = True

C = 10
gamma = 0.01

if load_clfs:
    clfs = load_pickle('pickles/clfs.pickle')
else:
    # Train model for each class
    clfs = []
    for col in range(y_train.shape[1]):
        print("Training model for class", col)
        clf = SVC(C=C, gamma=gamma).fit(X_train, y_train[:, col])
        clfs.append(clf)

    print()
    save_pickle('pickles/clfs.pickle', clfs)

## Part 3 - Fit classifiers into OneVsAll Classifier
### Note
If the one-vs-all classifier predicts multiple digits but is still correct for one of them, it is considered as an
accurate prediction.

In [11]:
# Train/load OneVsAll classifier and predict training samples.
ova_clf = a3.OneVsAllClassifier(clfs)

print("Calculating test errors for OneVsAll...")
y_pred_ova = ova_clf.predict(X_test)

# Calculate error count.
y_diff_ova = y_test - y_pred_ova
pred_count = int(y_test.shape[0])
error_ova = np.where(y_diff_ova == 1)[0].shape[0]  # False negatives are 1.
print(f"{error_ova} errors out of {pred_count} predictions")

# Calculate accuracy.
accuracy_ova = round((pred_count - error_ova) / pred_count, 3)
print(f"Accuracy = {accuracy_ova*100}%")

Calculating test errors for OneVsAll...
709 errors out of 10000 predictions
Accuracy = 92.9%


## Part 4 - Train sklearn one-vs-one SVC

In [12]:
# Reformat y from 2d to 1d to work with sklearn one-vs-one classifier.
y_train_ovo = np.zeros(y_train.shape[0])
for i in range(y_train.shape[0]):
    y_train_ovo[i] = np.where(y_train[i] == 1)[0][0]

print("Training OneVsOne classifier")
# Train/load OneVsOne classifier and predict training samples.
ovo_clf = OneVsOneClassifier(SVC(C=C, gamma=gamma)).fit(X_train, y_train_ovo)
print("Done...\n")

y_pred_ovo = ovo_clf.predict(X_test)

print("Calculating test errors for OneVsOne...")

# Reformat y from 2d to 1d to match sklearn one-vs-one classifier standard.
y_test_ovo = np.zeros(y_test.shape[0])
for i in range(y_test.shape[0]):
    y_test_ovo[i] = np.where(y_test[i] == 1)[0][0]

# Error count.
y_diff_ovo = (y_test_ovo != y_pred_ovo).astype(int)
error_ovo = np.sum(y_diff_ovo)
print(f"{error_ovo} errors out of {pred_count} predictions")

# Calculate accuracy.
accuracy_ovo = round((pred_count - error_ovo) / pred_count, 3)
print(f"Accuracy = {accuracy_ovo*100}%")

Training OneVsOne classifier
Done...

Calculating test errors for OneVsOne...
349 errors out of 10000 predictions
Accuracy = 96.5%


## Part 5 - Evaluation of multi-class classification strategies
### Confusion matrix

In [49]:
print("Columns: real response.")
print("Rows: predicted response.\n")

# OneVsAll
matrix_ova = np.zeros((10,10))
for digit in range(len(matrix_ova[0])):
    y_digit = y_test[:, digit]
    y_pred_digit = y_pred_ova[np.where(y_digit == 1)[0]]
    matrix_col = np.sum(y_pred_digit, axis=0)
    matrix_ova[:, digit] = matrix_col

print("OneVsAll confusion matrix")
print(tabulate(matrix_ova, headers=np.arange(10), showindex="always"))

# OneVsOne
matrix_ovo = np.zeros((10,10))
for digit in range(len(matrix_ovo[0])):
    y_pred_digit = y_pred_ovo[np.where(y_test_ovo == digit)[0]]
    digits, counts = np.unique(y_pred_digit, return_counts=True)
    matrix_col = np.zeros(10)
    for i in range(10):
        if i in digits:
            matrix_col[i] = counts[np.where(digits == i)[0]]

    matrix_ovo[:, digit] = matrix_col

print("\nOneVsOne confusion matrix")
print(tabulate(matrix_ovo, headers=np.arange(10), showindex="always"))

Columns: real response.
Rows: predicted response.

OneVsAll confusion matrix
      0     1    2    3    4    5    6    7    8    9
--  ---  ----  ---  ---  ---  ---  ---  ---  ---  ---
 0  955     0    2    0    0    5    3    0    0    1
 1    0  1113    2    0    0    0    3    5    2    3
 2    1     2  951    2    2    0    0   10    3    0
 3    0     2    2  926    0    3    0    0   10    1
 4    1     0    5    1  928    2    7    4    2   18
 5    3     0    0   10    0  814    8    0    4    2
 6    2     2    6    1    6    6  905    0    1    1
 7    1     0   10    8    1    1    0  944    2    6
 8    3     1    8    7    4    7    2    0  857    0
 9    2     0    4    3   24    1    0   20    0  898

OneVsOne confusion matrix
      0     1    2    3    4    5    6    7    8    9
--  ---  ----  ---  ---  ---  ---  ---  ---  ---  ---
 0  967     0    6    0    1    6    4    0    3    4
 1    0  1125    0    0    0    1    3    8    1    6
 2    2     3  999   12    3    

### Evaluation

Overall, both classifiers perform quite well. The test results show that:
1. OneVsAll has an accuracy of 92.9% whereas OneVsOne has an accuracy of 96.5%
2. OneVsOne had nearly half the number of prediction errors for the tests ->
    OneVsOne had 349 errors and OneVsAll had 709.

The confusion matrices appear to be quite similar for both classifiers with similar patterns of
errors. Digits that in nature appear similar get predicted incorrectly more often than those
that do not. Some examples are:
* 4 predicted as 9.
* 7 predicted as 9.
* 8 predicted as 3.

Variance does exist between the two classifiers, however. OneVsOne classifies 5s as 3s and 3s as 2s
significantly more often than OneVsAll.

Picking a classifier, therefore, becomes a situational matter. If predicting 5s as 3s is
a more costful error to the user than having less overall accuracy, the user may
choose to use the OneVsAll classifier. Likewise, if overall accuracy matters most to another user,
they may choose to use the OneVsOne classifier.

Finally, it is important to note that the OneVsOne classifier takes significantly longer to predict
than OneVsAll since it consists of more classifiers.
* The OneVsAll classifier is bundled with 10 classifiers.
* The OneVsOne classifier is bundled with 45 classifiers.

This means that the OneVsOne classifier has to predict 4.5x more than the OneVsAll classifier which is a significant
increase in required computational power.